<font size=6> The total sovereign emissions calculated by adding up scope 1, 2 and 3 emissions  </font>

In [1]:
report_currency = 'USD'

In [2]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

#trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [4]:
import boto3
import pandas as pd
import pint

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [5]:
def requantify_df(df):
    units_col = None
    columns_reversed = reversed(df.columns)
    for col in columns_reversed:
        if col.endswith("_units"):
            if units_col:
                # We expect _units column to follow a non-units column
                raise ValueError
            units_col = col
            continue
        if units_col:
            if col + '_units' != units_col:
                raise ValueError
            if (df[units_col]==df[units_col][0]).all():
                # Make a PintArray
                new_col = PintArray(df[col], dtype=f"pint[{ureg(df[units_col][0]).u}]")
            else:
                # Make a pd.Series of Quantity in a way that does not throw UnitStrippedWarning
                new_col = pd.Series(data=df[col], name=col) * pd.Series(data=df[units_col].map(lambda x: ureg(x).u), name=col)
            df = df.drop(columns=units_col)
            df[col] = new_col
            units_col = None
    return df


In [6]:
# If DF_COL contains Pint quantities (because it is a PintArray or an array of Pint Quantities),
# return a two-column dataframe of magnitudes and units.
# If DF_COL contains no Pint quanities, return it unchanged.

def dequantify_column(df_col: pd.Series):
    if type(df_col.values)==PintArray:
        return pd.DataFrame({df_col.name: df_col.values.quantity.m,
                             df_col.name + "_units": str(df_col.values.dtype.units)},
                            index=df_col.index)
    elif df_col.size==0:
        return df_col
    elif isinstance(df_col.iloc[0], Quantity):
        values = df_col.map(lambda x: (x.m, x.u))
        return pd.DataFrame({df_col.name: df_col.map(lambda x: x.m),
                             df_col.name + "_units": df_col.map(lambda x: str(x.u))},
                            index=df_col.index)
    else:
        return df_col

# Rewrite dataframe DF so that columns containing Pint quantities are represented by a column for the Magnitude and column for the Units.
# The magnitude column retains the original column name and the units column is renamed with a _units suffix.
def dequantify_df(df):
    return pd.concat([dequantify_column(df[col]) for col in df.columns], axis=1)

In [7]:
# Get the values saved in Trino related with Foreign CO2 emissions embodied in gross imports

import pandas as pd
import pandas as pd
from openscm_units import unit_registry
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType

ureg = unit_registry
Q_ = ureg.Quantity
ureg.default_format = '~'
ureg.define("CO2e = CO2 = CO2eq")
ureg.define("USD = [currency] ")
ureg.define("EUR = [currency_EUR] ")
ureg.define('Millions=1000000')
set_application_registry(ureg)


ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_total_sovereign_emissions'


src_table_1 = 'sf_oecd_imgr_fco2'
src_table_2 = 'sf_unfccc_results'
src_table_3 = 'sf_oecd_exgr_dco2'
src_table_4 = 'sf_oecd_exch_rates'
src_table_5 = 'sf_wdi_population'

sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1} """ 
df1 = pd.read_sql(sql, engine)
df1 = requantify_df(df1)
df1
# convert to CO2 units to t
df1['value'] = df1['value'].pint.to("t CO2e")
df1 = dequantify_df(df1)
df1

,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,GBR,ESP,DTOTAL,1995,Foreign CO2 emissions embodied in gross imports,6879000.0,CO2eq * t
1,GBR,ESP,DTOTAL,1996,Foreign CO2 emissions embodied in gross imports,7601000.0,CO2eq * t
2,GBR,ESP,DTOTAL,1997,Foreign CO2 emissions embodied in gross imports,8927000.0,CO2eq * t
3,GBR,ESP,DTOTAL,1998,Foreign CO2 emissions embodied in gross imports,10208000.0,CO2eq * t
4,GBR,ESP,DTOTAL,1999,Foreign CO2 emissions embodied in gross imports,11888000.0,CO2eq * t
...,...,...,...,...,...,...,...
334651,KOR,FRA,DTOTAL,2006,Foreign CO2 emissions embodied in gross imports,1506000.0,CO2eq * t
334652,KOR,FRA,DTOTAL,2007,Foreign CO2 emissions embodied in gross imports,1605000.0,CO2eq * t
334653,KOR,FRA,DTOTAL,2008,Foreign CO2 emissions embodied in gross imports,1767000.0,CO2eq * t
334654,KOR,FRA,DTOTAL,2009,Foreign CO2 emissions embodied in gross imports,1251000.0,CO2eq * t


In [8]:
sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_3} where industry_code='DTOTAL'""" 
df_exgr = pd.read_sql(sql, engine)
df_exgr = requantify_df(df_exgr)
# convert to CO2 units to t
df_exgr['value'] = df_exgr['value'].pint.to("t CO2e")
df_exgr = dequantify_df(df_exgr)
df_exgr

,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,ROU,WLD,DTOTAL,1995,Domestic CO2 emissions embodied in gross exports,28461000.0,CO2eq * t
1,ROU,WLD,DTOTAL,1996,Domestic CO2 emissions embodied in gross exports,28574000.0,CO2eq * t
2,ROU,WLD,DTOTAL,1997,Domestic CO2 emissions embodied in gross exports,28330000.0,CO2eq * t
3,ROU,WLD,DTOTAL,1998,Domestic CO2 emissions embodied in gross exports,21510000.0,CO2eq * t
4,ROU,WLD,DTOTAL,1999,Domestic CO2 emissions embodied in gross exports,20806000.0,CO2eq * t
...,...,...,...,...,...,...,...
167323,PER,ROU,DTOTAL,1998,Domestic CO2 emissions embodied in gross exports,1000.0,CO2eq * t
167324,PER,ROU,DTOTAL,1999,Domestic CO2 emissions embodied in gross exports,1000.0,CO2eq * t
167325,PER,ROU,DTOTAL,2000,Domestic CO2 emissions embodied in gross exports,3000.0,CO2eq * t
167326,PER,ROU,DTOTAL,2001,Domestic CO2 emissions embodied in gross exports,1000.0,CO2eq * t


In [9]:
###########################

sql=f"""
select validity_date,value from {ingest_catalog}.{ingest_schema}.{src_table_4} where country_iso_code='DEU'""" 
df_exch_eur = pd.read_sql(sql, engine)
df_exch_eur

,validity_date,value
0,1950,2.144861
1,1951,2.144861
2,1952,2.144861
3,1953,2.147426
4,1954,2.147426
...,...,...
67,2017,0.885206
68,2018,0.846773
69,2019,0.893276
70,2020,0.875506


In [10]:
import pandas as pd 
sql=f"""
select country_iso_code,validity_date,value from {ingest_catalog}.{ingest_schema}.{src_table_5} """ 
df_population = pd.read_sql(sql, engine)
df_population["value"] = pd.to_numeric(df_population["value"], errors='coerce')
df_population["validity_date"] = pd.to_numeric(df_population["validity_date"], errors='coerce')
df_population=df_population.convert_dtypes()
df_population
#df_population.info(verbose=True)


,country_iso_code,validity_date,value
0,AFG,2014,33370804
1,ALB,2014,2889104
2,DZA,2014,38923688
3,ASM,2014,55791
4,AND,2014,79213
...,...,...,...
3187,SSF,2012,917725880
3188,SSA,2012,917637577
3189,TSS,2012,917725880
3190,UMC,2012,2354967527


<font size=3> Remove invalid country codes </font>

In [11]:
import pycountry
import pandas as pd
df_country = pd.DataFrame([country.__dict__['_fields'] for country in pycountry.countries])
df_country = df_country.rename(columns={'alpha_3':'country_iso_code'})
df_country=df_country[['country_iso_code']]
# add Rest of World to the country dataframe
dict_row = {'country_iso_code':'ROW'}
df_country = pd.concat([df_country,pd.Series(dict_row)],ignore_index=True)
df_country.info(verbose=True)
df1=df1.convert_dtypes()
#df1.info(verbose=True)
df1 = pd.merge(df1,df_country,on=['country_iso_code'])  
df1=df1.convert_dtypes()
#df_result.drop(columns=['country_name_y'],inplace=True)
df1

df_exgr = pd.merge(df_exgr,df_country,on=['country_iso_code'])  
df_exgr=df_exgr.convert_dtypes()
#df_result.drop(columns=['country_name_y'],inplace=True)
df_exgr






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_iso_code  249 non-null    object
 1   0                 1 non-null      object
dtypes: object(2)
memory usage: 4.0+ KB


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units,0
0,ROU,WLD,DTOTAL,1995,Domestic CO2 emissions embodied in gross exports,28461000.0,CO2eq * t,<NA>
1,ROU,WLD,DTOTAL,1996,Domestic CO2 emissions embodied in gross exports,28574000.0,CO2eq * t,<NA>
2,ROU,WLD,DTOTAL,1997,Domestic CO2 emissions embodied in gross exports,28330000.0,CO2eq * t,<NA>
3,ROU,WLD,DTOTAL,1998,Domestic CO2 emissions embodied in gross exports,21510000.0,CO2eq * t,<NA>
4,ROU,WLD,DTOTAL,1999,Domestic CO2 emissions embodied in gross exports,20806000.0,CO2eq * t,<NA>
...,...,...,...,...,...,...,...,...
133051,BEL,MMR,DTOTAL,2014,Domestic CO2 emissions embodied in gross exports,4000.0,CO2eq * t,<NA>
133052,BEL,MMR,DTOTAL,2015,Domestic CO2 emissions embodied in gross exports,6000.0,CO2eq * t,<NA>
133053,BEL,MMR,DTOTAL,2016,Domestic CO2 emissions embodied in gross exports,9000.0,CO2eq * t,<NA>
133054,BEL,MMR,DTOTAL,2017,Domestic CO2 emissions embodied in gross exports,7000.0,CO2eq * t,<NA>


Calcuate Scope 2:
GHG emissions occurring as a consequence of the domestic use of grid-supplied electricity, heat,
steam and/or cooling which is imported from another territory


In [12]:
rename_columns = {'value':'total','partner_iso_code':'country_iso_code','value_units' :'total_units'}
agg_columns = { 'value' : 'sum'}
columns_order = ['attribute','industry_code','country_iso_code','validity_date','total','total_units']
df1 = df1.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df1.convert_dtypes()

df1.info(verbose=True)
df1

######################

df_exgr = df_exgr.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df_exgr.convert_dtypes()

df_exgr.info(verbose=True)
df_exgr



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         4032 non-null   string 
 1   industry_code     4032 non-null   string 
 2   country_iso_code  4032 non-null   string 
 3   validity_date     4032 non-null   Int64  
 4   total             4032 non-null   Float64
 5   total_units       4032 non-null   string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 197.0 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         2016 non-null   string 
 1   industry_code     2016 non-null   string 
 2   country_iso_code  2016 non-null   string 
 3   validity_date     2016 non-null   Int64  
 4   total             2016 non-null   Float64
 5   total_units    

,attribute,industry_code,country_iso_code,validity_date,total,total_units
0,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1995,1805713000.0,CO2eq * t
1,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1996,1841366000.0,CO2eq * t
2,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1997,1951115000.0,CO2eq * t
3,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1998,1983288000.0,CO2eq * t
4,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1999,2081499000.0,CO2eq * t
...,...,...,...,...,...,...
2011,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2014,225874000.0,CO2eq * t
2012,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2015,198104000.0,CO2eq * t
2013,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2016,178284000.0,CO2eq * t
2014,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2017,190538000.0,CO2eq * t


In [13]:
# divide the dataframe into two dataframes 
# to calculate Scope2 and Scope3 Values
df_D35 = df1[df1['industry_code']=='D35']
df_D35 = df_D35[['country_iso_code','validity_date','total','total_units']]
df_DTOTAL = df1[df1['industry_code']=='DTOTAL']
df_DTOTAL = df_DTOTAL[['country_iso_code','validity_date','total','total_units']]
df_result = pd.merge(df_D35,df_DTOTAL,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()

#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total_x':'scope2_value','total_y':'scope3_value','total_units_x':'scope2_value_units','total_units_y':'scope3_value_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result['scope3_value'] = df_result['scope3_value'] - df_result['scope2_value'] 
df_result[df_result['country_iso_code']=='CAN']

df_result = pd.merge(df_result,df_exgr,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()
#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total':'exported_emissions','total_units':'exported_emissions_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result.drop(columns=['attribute','industry_code'],inplace=True)
df_result.info(verbose=True)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   country_iso_code          2016 non-null   string 
 1   validity_date             2016 non-null   Int64  
 2   scope2_value              2016 non-null   Float64
 3   scope2_value_units        2016 non-null   string 
 4   scope3_value              2016 non-null   Float64
 5   scope3_value_units        2016 non-null   string 
 6   exported_emissions        2016 non-null   Float64
 7   exported_emissions_units  2016 non-null   string 
dtypes: Float64(3), Int64(1), string(4)
memory usage: 134.0 KB


In [14]:
# Get the values saved in Trino containing  GHG emissions provided by UNFCCC & GDP PPP values from Worldbank

import pandas as pd


sql=f"""
select country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_without_lulucf_units,scope1_excl_source,ghg_total_with_lulucf,ghg_total_with_lulucf_units
,gdp,gdp_units,gdp_ppp,gdp_ppp_units
from {ingest_catalog}.{ingest_schema}.{src_table_2} """ 
df_unfccc = pd.read_sql(sql, engine)

if (report_currency == 'EUR'):
    df_unfccc = pd.merge(df_unfccc,df_exch_eur,on=['validity_date'],how='inner')   
    df_unfccc['gdp'] = df_unfccc['gdp'] * df_unfccc['value']
    df_unfccc['gdp_ppp'] = df_unfccc['gdp_ppp'] * df_unfccc['value']
    df_unfccc['gdp_units'] = 'EUR'
    df_unfccc['gdp_ppp_units'] = 'EUR'
    df_unfccc.drop(columns=['value'],inplace=True)
    


df_unfccc[df_unfccc['country_iso_code']=='ARG']
df_unfccc = df_unfccc.convert_dtypes()
df_unfccc.info(verbose=True)
#df1["units"] = "kt"
#df1 = requantify_df(df1).convert_dtypes()
#df1.info(verbose=True)
#df1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15025 entries, 0 to 15024
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country_iso_code                15025 non-null  string 
 1   validity_date                   15025 non-null  Int64  
 2   ghg_total_without_lulucf        10633 non-null  Float64
 3   ghg_total_without_lulucf_units  15025 non-null  string 
 4   scope1_excl_source              10633 non-null  string 
 5   ghg_total_with_lulucf           10634 non-null  Float64
 6   ghg_total_with_lulucf_units     15025 non-null  string 
 7   gdp                             13118 non-null  Float64
 8   gdp_units                       15025 non-null  string 
 9   gdp_ppp                         7472 non-null   Float64
 10  gdp_ppp_units                   15025 non-null  string 
dtypes: Float64(4), Int64(1), string(6)
memory usage: 1.3 MB


In [15]:
import numpy as np
df_unfccc = df_unfccc.convert_dtypes()
df_unfccc.info(verbose=True)

df_unfccc = df_unfccc.assign(ghg_total_without_lulucf=np.where(df_unfccc.ghg_total_without_lulucf.isnull(), 0, df_unfccc.ghg_total_without_lulucf))
df_unfccc = df_unfccc.assign(ghg_total_with_lulucf=np.where(df_unfccc.ghg_total_with_lulucf.isnull(), 0, df_unfccc.ghg_total_with_lulucf))
df_unfccc = df_unfccc.assign(gdp=np.where(df_unfccc.gdp.isnull(), 0, df_unfccc.gdp))
df_unfccc = df_unfccc.assign(gdp_ppp=np.where(df_unfccc.gdp_ppp.isnull(), 0, df_unfccc.gdp_ppp))

df_unfccc = requantify_df(df_unfccc)
df_unfccc.info(verbose=True)
# convert to CO2 units to Mt
df_unfccc['ghg_total_without_lulucf'] = df_unfccc['ghg_total_without_lulucf'].pint.to("t CO2e")
df_unfccc['ghg_total_with_lulucf'] = df_unfccc['ghg_total_with_lulucf'].pint.to("t CO2e")
####

#df_unfccc = df_unfccc.assign(ghg_total_without_lulucf=np.where(df_unfccc.ghg_total_without_lulucf==0, np.nan, df_unfccc.ghg_total_without_lulucf))
#df_unfccc = df_unfccc.assign(ghg_total_with_lulucf=np.where(df_unfccc.ghg_total_with_lulucf==0, np.nan, df_unfccc.ghg_total_with_lulucf))
#df_unfccc = df_unfccc.assign(gdp=np.where(df_unfccc.gdp==0, np.nan, df_unfccc.gdp))
#df_unfccc = df_unfccc.assign(gdp_ppp=np.where(df_unfccc.gdp_ppp==0, np.nan, df_unfccc.gdp_ppp))

df_unfccc.info(verbose=True)


df_unfccc[df_unfccc['country_iso_code']=='ARG']



#df_unfccc['gdp'] = df_unfccc['gdp'].pint.to("Millions USD")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15025 entries, 0 to 15024
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country_iso_code                15025 non-null  string 
 1   validity_date                   15025 non-null  Int64  
 2   ghg_total_without_lulucf        10633 non-null  Float64
 3   ghg_total_without_lulucf_units  15025 non-null  string 
 4   scope1_excl_source              10633 non-null  string 
 5   ghg_total_with_lulucf           10634 non-null  Float64
 6   ghg_total_with_lulucf_units     15025 non-null  string 
 7   gdp                             13118 non-null  Float64
 8   gdp_units                       15025 non-null  string 
 9   gdp_ppp                         7472 non-null   Float64
 10  gdp_ppp_units                   15025 non-null  string 
dtypes: Float64(4), Int64(1), string(6)
memory usage: 1.3 MB
<class 'pandas.core.frame.DataFrame'>

/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(


,country_iso_code,validity_date,ghg_total_without_lulucf,scope1_excl_source,ghg_total_with_lulucf,gdp,gdp_ppp
504,ARG,1979,234376590.0,PRIMAP,58628235.0,69252328951.6441,0.0
505,ARG,1980,230395230.0,PRIMAP,58825652.0,76961923741.0701,0.0
506,ARG,1981,221494530.0,PRIMAP,59097876.0,78676842366.5529,0.0
507,ARG,1982,220520900.0,PRIMAP,86602885.0,84307486836.724,0.0
4437,ARG,1972,207027680.0,PRIMAP,60974355.0,34733000536.2862,0.0
...,...,...,...,...,...,...,...
13008,ARG,2007,328013980.0,PRIMAP,113335290.0,287530508430.568,658372466794.536
13009,ARG,2008,330940470.0,PRIMAP,90416481.0,361558037110.419,698223627731.022
13010,ARG,2009,309791380.0,PRIMAP,66680994.99999999,332976484577.619,661109513023.278
13011,ARG,2011,330382790.0,PRIMAP,55827120.0,530163281574.658,797263881242.037


In [16]:
#df_unfccc = dequantify_df(df_unfccc)
#df_unfccc
#df_unfccc.info(verbose=True)
#df_unfccc

df_result= requantify_df(df_result)
#df_result= dequantify_df(df_result)
#df_result


/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(


<font size=3><b>Merge UNFCCC , Worldbank and OECD data</b> </font>

In [17]:
#df_result.info(verbose=True)
#df_result= requantify_df(df_result)
#df_result.info(verbose=True)

#df_unfccc = df_unfccc.assign(ghg_total_without_lulucf=np.where(df_unfccc.ghg_total_without_lulucf.isnull(), 0, df_unfccc.ghg_total_without_lulucf))
#df_unfccc = df_unfccc.assign(ghg_total_with_lulucf=np.where(df_unfccc.ghg_total_with_lulucf.isnull(), 0, df_unfccc.ghg_total_with_lulucf))
#df_unfccc = df_unfccc.assign(gdp=np.where(df_unfccc.gdp.isnull(), 0, df_unfccc.gdp))
#df_unfccc = df_unfccc.assign(gdp_ppp=np.where(df_unfccc.gdp_ppp.isnull(), 0, df_unfccc.gdp_ppp))




df_result = pd.merge(df_result,df_unfccc,on=['country_iso_code','validity_date'],how='outer')  


rename_columns = {'ghg_total_without_lulucf':'scope1_excl_lulucf',
                  'ghg_total_with_lulucf':'scope1_incl_lulucf'}
#columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_excl_lulucf_units','scope1_incl_lulucf','scope1_incl_lulucf_units','scope2_value','scope2_value_units','scope3_value','scope3_value_units','exported_emissions','exported_emissions_units','gdp_ppp','gdp_ppp_units','gdp','gdp_units']
df_result = df_result.rename(columns=rename_columns)
df_result.info(verbose=True)

# enrich with population data

df_result = pd.merge(df_result,df_population,on=['country_iso_code','validity_date'],how='outer')  


rename_columns = {'value':'population'}
#columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_excl_lulucf_units','scope1_incl_lulucf','scope1_incl_lulucf_units','scope2_value','scope2_value_units','scope3_value','scope3_value_units','exported_emissions','exported_emissions_units','gdp_ppp','gdp_ppp_units','gdp','gdp_units']
df_result = df_result.rename(columns=rename_columns)
df_result.info(verbose=True)
df_result["population"] = df_result["population"].fillna(0)

df_result = dequantify_df(df_result)

#df_result['scope1_excl_lulucf'] = df_result['scope1_excl_lulucf'].fillna(0).astype(float)
#df_result['scope1_incl_lulucf'] = df_result['scope1_incl_lulucf'].fillna(0).astype(float)
#df_result['scope1_excl_lulucf'] = df_result['scope1_excl_lulucf'].fillna(0).astype(float)
#df_result['scope2_value'] = df_result['scope2_value'].fillna(0).astype(float)
#df_result['scope3_value'] = df_result['scope3_value'].fillna(0).astype(float)



df_result['sum_yn'] = 1 * ((df_result['scope1_excl_lulucf'] > 0) & (df_result['scope2_value'] > 0) &  (df_result['scope3_value'] > 0) )
df_result['calc_attr_factor_yn'] = 1 * (df_result['gdp_ppp'] > 0)
df_result['population_available'] = 1 * (df_result['population'] > 0)


df_result['consumption_emissions_excl_lulucf'] = df_result['sum_yn'] * ( df_result['scope1_excl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value'] - df_result['exported_emissions'])
df_result['consumption_emissions_incl_lulucf'] = df_result['sum_yn'] * ( df_result['scope1_incl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value'] - df_result['exported_emissions'])

df_result.info(verbose=True)

df_result['consumption_emissions_excl_lulucf_per_capita']  = df_result['population_available'] * df_result['consumption_emissions_excl_lulucf'] / df_result['population']
df_result['consumption_emissions_incl_lulucf_per_capita']  = df_result['population_available'] * df_result['consumption_emissions_incl_lulucf'] / df_result['population']


###############
df_result['attribution_factor_scope1_excl_lulucf']= df_result['calc_attr_factor_yn'] * (df_result['scope1_excl_lulucf']) / df_result['gdp_ppp']
df_result['attribution_factor_scope1_incl_lulucf']= df_result['calc_attr_factor_yn'] * (df_result['scope1_incl_lulucf']) / df_result['gdp_ppp']

df_result['attribution_factor_excl_lulucf']= df_result['calc_attr_factor_yn'] * df_result['sum_yn'] * ( (df_result['scope1_excl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp_ppp'])
df_result['attribution_factor_incl_lulucf']= df_result['calc_attr_factor_yn'] * df_result['sum_yn'] * ( (df_result['scope1_incl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp_ppp'])
##########
df_result['attribution_factor_excl_lulucf_gdp']= df_result['calc_attr_factor_yn'] * df_result['sum_yn'] * ( (df_result['scope1_excl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp'])
df_result['attribution_factor_incl_lulucf_gdp']= df_result['calc_attr_factor_yn'] * df_result['sum_yn'] * ( (df_result['scope1_incl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp'])



#df_result['ghg_total_without_lulucf']=df_result['ghg_total_without_lulucf'].replace(0, np.nan)
#df_result['ghg_total_with_lulucf']=df_result['ghg_total_with_lulucf'].replace(0, np.nan)
#df_result['gdp']=df_result['gdp'].replace(0, np.nan)
#df_result['gdp_ppp']=df_result['gdp_ppp'].replace(0, np.nan)



#df_result

columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_excl_source','scope1_incl_lulucf','scope2_value','scope3_value',
                 'exported_emissions','gdp_ppp','gdp',
                 'consumption_emissions_excl_lulucf','consumption_emissions_incl_lulucf',
                 'population','consumption_emissions_excl_lulucf_per_capita','consumption_emissions_incl_lulucf_per_capita',
                 'attribution_factor_scope1_excl_lulucf','attribution_factor_scope1_incl_lulucf',
                 'attribution_factor_excl_lulucf','attribution_factor_excl_lulucf_gdp',
                 'attribution_factor_incl_lulucf','attribution_factor_incl_lulucf_gdp']
df_result = df_result.reindex(columns=columns_order)
df_result.info(verbose=True)
df_result= dequantify_df(df_result)
df_result.info(verbose=True)
# remove invalid country_iso_codes (regions, ...)

df_result = pd.merge(df_result,df_country,on=['country_iso_code'])  

cols= ['scope1_excl_lulucf','scope1_incl_lulucf','scope2_value','scope3_value',
                 'exported_emissions','gdp_ppp','gdp',
                 'consumption_emissions_excl_lulucf','consumption_emissions_incl_lulucf',
                 'population','consumption_emissions_excl_lulucf_per_capita','consumption_emissions_incl_lulucf_per_capita',
                 'attribution_factor_scope1_excl_lulucf','attribution_factor_scope1_incl_lulucf',
                 'attribution_factor_excl_lulucf','attribution_factor_excl_lulucf_gdp',
                 'attribution_factor_incl_lulucf','attribution_factor_incl_lulucf_gdp']
df_result[cols] = df_result[cols].replace({0:np.nan})

df_result.info(verbose=True)

df_result


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15433 entries, 0 to 15432
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   country_iso_code    15433 non-null  string         
 1   validity_date       15433 non-null  Int64          
 2   scope2_value        2016 non-null   pint[CO2eq * t]
 3   scope3_value        2016 non-null   pint[CO2eq * t]
 4   exported_emissions  2016 non-null   pint[CO2eq * t]
 5   scope1_excl_lulucf  15025 non-null  pint[CO2eq * t]
 6   scope1_excl_source  10633 non-null  string         
 7   scope1_incl_lulucf  15025 non-null  pint[CO2eq * t]
 8   gdp                 15025 non-null  pint[USD]      
 9   gdp_ppp             15025 non-null  pint[USD]      
dtypes: Int64(1), pint[CO2eq * t](5), pint[USD](2), string(2)
memory usage: 1.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15536 entries, 0 to 15535
Data columns (total 11 columns):
 #   Column    

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_source,scope1_incl_lulucf,scope2_value,scope3_value,exported_emissions,gdp_ppp,gdp,...,population,consumption_emissions_excl_lulucf_per_capita,consumption_emissions_incl_lulucf_per_capita,attribution_factor_scope1_excl_lulucf,attribution_factor_scope1_incl_lulucf,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_gdp,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_gdp,0
0,ARG,1995,250822500.0,PRIMAP,6.843166e+07,21000.0,7923000.0,17594000.0,3.468447e+11,2.580318e+11,...,<NA>,NaN,NaN,0.000723,0.000197,0.000746,0.001003,0.000220,0.000296,NaN
1,ARG,1996,264400040.0,PRIMAP,8.749583e+07,28000.0,9506000.0,19889000.0,3.727153e+11,2.721498e+11,...,<NA>,NaN,NaN,0.000709,0.000235,0.000735,0.001007,0.000260,0.000357,NaN
2,ARG,1997,265184700.0,PRIMAP,2.419562e+08,24000.0,12937000.0,24059000.0,4.098950e+11,2.928590e+11,...,<NA>,NaN,NaN,0.000647,0.000590,0.000679,0.000950,0.000622,0.000870,NaN
3,ARG,1998,271059620.0,PRIMAP,1.012612e+08,43000.0,11260000.0,25197000.0,4.304678e+11,2.989482e+11,...,<NA>,NaN,NaN,0.000630,0.000235,0.000656,0.000945,0.000261,0.000377,NaN
4,ARG,1999,276703800.0,PRIMAP,8.667607e+07,46000.0,10479000.0,22794000.0,4.217554e+11,2.835230e+11,...,<NA>,NaN,NaN,0.000656,0.000206,0.000681,0.001013,0.000230,0.000343,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12292,GIB,2015,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,...,33742,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12293,GIB,2016,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,...,33738,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12294,GIB,2019,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,...,33706,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12295,GIB,2020,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,...,33691,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#df_result['scope1_value_units'] = 'Mt CO2e'
#df_result['scope1_value'] = df_result['scope1_value'].round(decimals = 3)
#pd.options.display.float_format = '{:.3f}'.format
#df_result= df_result.convert_dtypes()
#df_result.info(verbose=True)
df_result[df_result['country_iso_code']=='CAN']
#df_result

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_source,scope1_incl_lulucf,scope2_value,scope3_value,exported_emissions,gdp_ppp,gdp,...,population,consumption_emissions_excl_lulucf_per_capita,consumption_emissions_incl_lulucf_per_capita,attribution_factor_scope1_excl_lulucf,attribution_factor_scope1_incl_lulucf,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_gdp,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_gdp,0
414,CAN,1995,6.454553e+08,UNFCCC,6.058811e+08,5109000.0,172269000.0,129418000.0,NaN,6.040316e+11,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,CAN,1996,6.667019e+08,UNFCCC,6.212992e+08,5249000.0,180997000.0,130376000.0,NaN,6.285464e+11,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
416,CAN,1997,6.822990e+08,UNFCCC,6.329551e+08,6340000.0,195339000.0,151956000.0,7.553902e+11,6.549870e+11,...,<NA>,NaN,NaN,0.000903,0.000838,0.001170,0.001350,0.001105,0.001274,NaN
417,CAN,1998,6.886139e+08,UNFCCC,6.335611e+08,7969000.0,205395000.0,155181000.0,7.936743e+11,6.340000e+11,...,<NA>,NaN,NaN,0.000868,0.000798,0.001136,0.001423,0.001067,0.001336,NaN
418,CAN,1999,7.017587e+08,UNFCCC,6.540323e+08,9069000.0,214154000.0,158931000.0,8.464129e+11,6.784122e+11,...,<NA>,NaN,NaN,0.000829,0.000773,0.001093,0.001363,0.001036,0.001293,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,CAN,1972,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,1.130828e+11,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472,CAN,1980,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,2.738538e+11,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,CAN,1964,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,4.937752e+10,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,CAN,2019,7.382834e+08,UNFCCC,7.223483e+08,NaN,NaN,NaN,1.853711e+12,1.742015e+12,...,37601230,<NA>,<NA>,0.000398,0.000390,NaN,NaN,NaN,NaN,NaN


Calculate Attribution factor = (Scope1+Scope2+Scope3)/GDP


In [19]:


#df_result= requantify_df(df_result)
format_mapper =  {'gdp_ppp': '{0:,.0f}',
           'gdp': '{0:,.0f}',       
           'attribution_factor_excl_lulucf': '{0:.8f}',
           'attribution_factor_incl_lulucf': '{0:.8f}',
           'attribution_factor_excl_lulucf_gdp': '{0:.8f}',
           'attribution_factor_incl_lulucf_gdp': '{0:.8f}',       
           'scope1_excl_lulucf': '{0:,.0f}',       
           'scope1_incl_lulucf': '{0:,.0f}',              
           'scope2_value': '{0:,.0f}',              
           'scope3_value': '{0:,.0f}'
                 }

df_result[df_result['country_iso_code']=='CAN'].style.format(format_mapper)


,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_source,scope1_incl_lulucf,scope2_value,scope3_value,exported_emissions,gdp_ppp,gdp,consumption_emissions_excl_lulucf,consumption_emissions_incl_lulucf,population,consumption_emissions_excl_lulucf_per_capita,consumption_emissions_incl_lulucf_per_capita,attribution_factor_scope1_excl_lulucf,attribution_factor_scope1_incl_lulucf,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_gdp,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_gdp,0
414,CAN,1995,"645,455,297",UNFCCC,"605,881,079","5,109,000","172,269,000",129418000.000000,nan,"604,031,623,433",693415296.776335,653841079.122202,,nan,nan,nan,nan,nan,nan,nan,nan,nan
415,CAN,1996,"666,701,910",UNFCCC,"621,299,195","5,249,000","180,997,000",130376000.000000,nan,"628,546,387,972",722571910.230366,677169195.292942,,nan,nan,nan,nan,nan,nan,nan,nan,nan
416,CAN,1997,"682,298,998",UNFCCC,"632,955,107","6,340,000","195,339,000",151956000.000000,"755,390,170,461","654,986,999,856",732021998.432811,682678106.804654,,nan,nan,0.000903,0.000838,0.00117023,0.00134961,0.00110490,0.00127428,nan
417,CAN,1998,"688,613,931",UNFCCC,"633,561,120","7,969,000","205,395,000",155181000.000000,"793,674,327,704","634,000,000,000",746796931.024858,691744119.917268,,nan,nan,0.000868,0.000798,0.00113646,0.00142268,0.00106709,0.00133584,nan
418,CAN,1999,"701,758,712",UNFCCC,"654,032,317","9,069,000","214,154,000",158931000.000000,"846,412,945,810","678,412,196,271",766050711.728973,718324316.832600,,nan,nan,0.000829,0.000773,0.00109283,0.00136345,0.00103644,0.00129310,nan
419,CAN,2000,"726,987,283",UNFCCC,"690,491,255","18,161,000","227,658,000",164946000.000000,"900,996,986,801","744,773,415,932",807860282.993296,771364254.730039,30685730,26.326904,25.137556,0.000807,0.000766,0.00107970,0.00130618,0.00103919,0.00125717,nan
420,CAN,2001,"718,320,043",UNFCCC,"673,786,320","19,495,000","219,866,000",161236000.000000,"937,786,776,177","738,981,792,355",796445043.152663,751911319.668803,,nan,nan,0.000766,0.000718,0.00102121,0.00129595,0.00097373,0.00123568,nan
421,CAN,2002,"724,281,130",UNFCCC,"714,722,119","8,161,000","224,333,000",150981000.000000,"971,003,788,233","760,649,334,098",805794129.555186,796235118.836829,,nan,nan,0.000746,0.000736,0.00098535,0.00125784,0.00097550,0.00124527,nan
422,CAN,2003,"743,286,854",UNFCCC,"733,290,060","8,188,000","223,717,000",161485000.000000,"1,023,682,523,003","895,540,646,635",813706854.384847,803710059.638611,,nan,nan,0.000726,0.000716,0.00095263,0.00108894,0.00094287,0.00107778,nan
423,CAN,2004,"745,194,047",UNFCCC,"733,509,090","8,378,000","229,556,000",176007000.000000,"1,083,609,294,356","1,026,690,238,278",807121047.462783,795436090.099890,,nan,nan,0.000688,0.000677,0.00090727,0.00095757,0.00089649,0.00094619,nan


<font size=3>Save the results in Trino</font>

In [20]:
# Add the sources used for the calculations in the result table
#df_result['scope1_excl_source'] = "UNFCCC -> Time_Series_GHG_total_without_LULUCF_in_kt_CO2_equivalent.xlsx"
df_result['scope1_incl_source'] = "UNFCCC -> Time_Series_GHG_total_with_LULUCF_in_kt_CO2_equivalent.xlsx"
df_result['scope2_source'] = "OECD -> IMGR_FCO2.csv -> Industry_Code = D35  -> Aggregation grouped by Partner_ISO_CODE"
df_result['scope3_source'] = "OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL - Scope_2 Value "          
df_result['gdp_ppp_source'] = "Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4019306.csv"          


In [21]:
import osc_ingest_trino as osc
#df_result['validity_date']=pd.to_datetime(df_result['validity_date'], unit='D')
df_result=df_result.convert_dtypes()
df_result.info(verbose=True)

df_result['year'] = df_result['validity_date']
#df_result['validity_date'] = pd.to_datetime(df_result.validity_date, format='%Y')
df_result = df_result.convert_dtypes()
#df_result.validity_date = df_result.validity_date.astype('datetime64[ns]')



columnschema = osc.create_table_schema_pairs(df_result,typemap={'datetime64[ns]':'timestamp(6)'}) 
sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
#print(qres.fetchall())





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12297 entries, 0 to 12296
Data columns (total 26 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   country_iso_code                              12297 non-null  string 
 1   validity_date                                 12297 non-null  Int64  
 2   scope1_excl_lulucf                            10633 non-null  Float64
 3   scope1_excl_source                            10633 non-null  string 
 4   scope1_incl_lulucf                            9329 non-null   Float64
 5   scope2_value                                  1368 non-null   Float64
 6   scope3_value                                  1584 non-null   Int64  
 7   exported_emissions                            1584 non-null   Int64  
 8   gdp_ppp                                       5942 non-null   Float64
 9   gdp                                           10312 non-null 

In [22]:
df_result.info("verbose=True")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12297 entries, 0 to 12296
Data columns (total 27 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   country_iso_code                              12297 non-null  string 
 1   validity_date                                 12297 non-null  Int64  
 2   scope1_excl_lulucf                            10633 non-null  Float64
 3   scope1_excl_source                            10633 non-null  string 
 4   scope1_incl_lulucf                            9329 non-null   Float64
 5   scope2_value                                  1368 non-null   Float64
 6   scope3_value                                  1584 non-null   Int64  
 7   exported_emissions                            1584 non-null   Int64  
 8   gdp_ppp                                       5942 non-null   Float64
 9   gdp                                           10312 non-null 

In [23]:
df_result = df_result[df_result['year']>=1990]
df_result.sort_values(by=['country_iso_code','validity_date'], inplace=True)
df_result.to_excel("pcaf_results.xlsx",index=False)
df_result.to_csv("pcaf_results.csv",index=False)


In [24]:
df_result.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 50 , verbose = True))


constructed fully qualified table name as: "pcaf_sovereign_footprint.sf_total_sovereign_emissions"
inserting 50 records
  ('ABW', 1990, NULL, NULL, NULL, NULL, NULL, NULL, 1363755808.78007, 764804469.273743, NULL, NULL, 62152, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'UNFCCC -> Time_Series_GHG_total_with_LULUCF_in_kt_CO2_equivalent.xlsx', 'OECD -> IMGR_FCO2.csv -> Industry_Code = D35  -> Aggregation grouped by Partner_ISO_CODE', 'OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL - Scope_2 Value ', 'Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4019306.csv', 1990)
  ('ABW', 1991, NULL, NULL, NULL, NULL, NULL, NULL, 1522141309.42665, 872067039.106145, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'UNFCCC -> Time_Series_GHG_total_with_LULUCF_in_kt_CO2_equivalent.xlsx', 'OECD -> IMGR_FCO2.csv -> Industry_Code = D35  -> Aggregation grouped by Partner_ISO_CODE', 'OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL - Scope_2 Value ', 'Worldbank -> API_NY.GDP.MKT

In [25]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table} where country_iso_code = 'BRA' and year > 2017 """
pd.read_sql(sql, engine)

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_source,scope1_incl_lulucf,scope2_value,scope3_value,exported_emissions,gdp_ppp,gdp,...,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_gdp,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_gdp,0,scope1_incl_source,scope2_source,scope3_source,gdp_ppp_source,year
0,BRA,2018,1.085199e+09,PRIMAP,389250400.0,7000.0,102481000.0,89875000.0,3.145953e+12,1.916934e+12,...,0.000378,0.00062,0.000156,0.000257,None,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...,2018
1,BRA,2019,1.099705e+09,PRIMAP,399492420.0,NaN,NaN,NaN,3.241317e+12,1.873288e+12,...,NaN,NaN,NaN,NaN,None,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...,2019
2,BRA,2020,1.082858e+09,PRIMAP,359934270.0,NaN,NaN,NaN,3.153147e+12,1.448566e+12,...,NaN,NaN,NaN,NaN,None,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...,2020
3,BRA,2021,1.132351e+09,PRIMAP,358955760.0,NaN,NaN,NaN,3.435882e+12,1.608981e+12,...,NaN,NaN,NaN,NaN,None,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...,2021
